In [11]:
import ipywidgets as ipw
from ipycanvas import Canvas, hold_canvas
import math
import time

ATTRIBUTE_TYPES = ['Atk','Def','Buff']

MAX_RUNES = 10
MAX_PER_ATT = {
    'Atk':MAX_RUNES,
    'Def':MAX_RUNES,
    'Buff':8,
}

REQUIRED_T1 = [0,1,1,1,2,3,3]
REQUIRED_F1 = [0,1,1,2,2,3,3]
REQUIRED_T2 = [0,2,3,3,4,5,6]
REQUIRED_F2 = [0,2,3,4,5,6,6]
REQUIRED_T3 = [0,3,5,6,8,9,10]
REQUIRED_F3 = [0,3,6,9,11,11,11]

ERROR = {
    'ERR_OK': 0,
    'ERR_MULT_UN_ATT_EXC': 1,
    'ERR_SING_UN_ATT_EXC': 2,
    'ERR_MULT_CD_ATT_EXC': 3,
    'ERR_SUM_EXC': 4
}

ERR_UNEXPECTED = 'Error {0} no esperado.'

MC_WRN_CENTER = 'Los mínimos marcados con * son mutuamente excluyentes por requerir la runa central, el valor entre () indica el mínimo sin usar la runa central.'
MC_ERR_SUM = '¡Irresoluble! Se necesitarían más de {0} runas de atributo para poder activar todas tus runas.'
MC_ERR_ATT = '¡Demasiadas runas de {0}! Solo hay {1} runas de {0}, pero activar todas tus runas requiere más de {1}.'
MC_SHOW = ipw.Layout(width='850px',visibility='visible')
MC_HIDE = ipw.Layout(width='850px',visibility='hidden')
TC_READY = 'Listo.'
TC_CALC = 'Calculando...'
TC_FAIL = 'Irresoluble.'

def reverseLookup(d,v):
    return list(d.keys())[list(d.values()).index(v)]

def main():
    required = dict([(t,[0]*6) for t in ATTRIBUTE_TYPES])
    minIfCentre = dict([(t,{True:0,False:0})for t in ATTRIBUTE_TYPES])
    targetVals = dict.fromkeys(ATTRIBUTE_TYPES,0)
    attributeError =ERROR['ERR_OK']
    prioTypes = dict.fromkeys(ATTRIBUTE_TYPES,True)
    reduceIfCenter = dict.fromkeys(ATTRIBUTE_TYPES,False)
    
    def req2List(i):
        l = list(map(lambda t: required[t][i],ATTRIBUTE_TYPES))
        return l

    def tryTake(l,t,n):
        i = ATTRIBUTE_TYPES.index(t)
        if sum(l) < n:
            return (True,l)
        elif l[i] == 0:
            return (False,l)
        else:
            c = l.copy()
            c[i] -= 1
            return (True,c)
    
    def calcPrio(first,dual,second):
        if dual:
            return [first,second,3-first-second]
        else:
            return list(range(3)) if first==0 else [first,0,3-first]
    
    def reduceToConstrain(vals,mins,maxSum,prio):
        if sum(vals) > maxSum:
            if vals[prio[0]] + vals[prio[1]] + mins[prio[2]] <= maxSum:
                vals[prio[2]] = maxSum - vals[prio[0]] - vals[prio[1]]
            else:
                vals[prio[2]] = mins[prio[2]]
                if vals[prio[0]] + mins[prio[1]] + mins[prio[2]] <= maxSum:
                    vals[prio[1]] = maxSum - vals[prio[0]] - mins[prio[2]]
                else:
                    vals[prio[1]] = mins[prio[1]]
                    vals[prio[0]] = maxSum - mins[prio[1]] - mins[prio[2]]
        
    
    def checkMin(req):
        
        semi_sum = sum(req)/2
        n_1 = sum(map(lambda x: 0 if x < 1 else 1, req))
        n_2 = sum(map(lambda x: 0 if x < 2 else 1, req))
        n_3 = req.count(3)
        
        return {True:max(math.floor(semi_sum),
                    REQUIRED_T1[n_1],
                    REQUIRED_T2[n_2],
                    REQUIRED_T3[n_3]),
                False:max(math.ceil(semi_sum),
                    REQUIRED_F1[n_1],
                    REQUIRED_F2[n_2],
                    REQUIRED_F3[n_3])}
    def classifyOptions():
        nonlocal attributeError, prioTypes, reduceIfCenter
        
        attributeError = ERROR['ERR_OK']
        prioTypes = dict.fromkeys(ATTRIBUTE_TYPES,True)
        
        reduction = dict(map(lambda t: (t,minIfCentre[t][False] - minIfCentre[t][True]),ATTRIBUTE_TYPES))
        reduceIfCenter = dict(map(lambda t: (t,reduction[t] > 0),ATTRIBUTE_TYPES))
        
        attExcess = dict(map(lambda t: (t,minIfCentre[t][True] > MAX_PER_ATT[t]),ATTRIBUTE_TYPES))
        nAttEx = list(attExcess.values()).count(True)
        
        attExcessIf = dict(map(lambda t: (t,minIfCentre[t][False] > MAX_PER_ATT[t]),ATTRIBUTE_TYPES))
        nAttExIf = list(attExcessIf.values()).count(True)
        
        sumIfNone = sum(map(lambda x: x[False],minIfCentre.values()))
        sumExcessIf = dict(map(lambda t: (t,sumIfNone - reduction[t] > MAX_RUNES ),ATTRIBUTE_TYPES))
        
        if nAttEx > 1:
            attributeError = ERROR['ERR_MULT_UN_ATT_EXC']
        elif nAttEx == 1:
            attributeError = ERROR['ERR_SING_UN_ATT_EXC']
            prioTypes = attExcess
        elif nAttExIf > 1:
            attributeError = ERROR['ERR_MULTI_CD_ATT_EXC']
        elif nAttExIf == 1:
            prioTypes = attExcessIf
            centerType = reverseLookup(attExcessIf,True)
            if sumExcessIf[centerType]:
                attributeError = ERROR['ERR_SUM_EXC']
        else:
            prioTypes = dict(map(lambda t: (t,not(sumExcessIf[t])),ATTRIBUTE_TYPES))
            nCenterCandidates = list(prioTypes.values()).count(True)
            if nCenterCandidates == 0:
                attributeError = ERROR['ERR_SUM_EXC']
        
    def constrainTargets(changedType):
        for t in ATTRIBUTE_TYPES:
            if targetVals[t] > MAX_PER_ATT[t]:
                 targetVals[t] = MAX_PER_ATT[t]
            elif targetVals[t] < minIfCentre[t][prioTypes[t]]:
                 targetVals[t] = minIfCentre[t][prioTypes[t]]
                    
        centerReq = dict(map(lambda t: (t,targetVals[t] < minIfCentre[t][False]),ATTRIBUTE_TYPES))
        nCenter = list(centerReq.values()).count(True)
        changedIndex = ATTRIBUTE_TYPES.index(changedType)
        
        if nCenter > 1:
            if centerReq[changedType]:
                for t in ATTRIBUTE_TYPES:
                    if centerReq[t] and t != changedType:
                        centerReq[t] = False
                        targetVals[t] = minIfCentre[t][False]
                prio = calcPrio(changedIndex,False,0)
            else:
                prio = calcPrio(changedIndex,False,0)
                t = ATTRIBUTE_TYPES[prio[2]]
                targetVals[t] = minIfCentre[t][False]
                centerReq[t] = False
        elif nCenter == 1:
            if centerReq[changedType]:
                prio = calcPrio(changedIndex,False,0)
            else:
                prio = calcPrio(changedIndex,True,ATTRIBUTE_TYPES.index(reverseLookup(centerReq,True)))
        else:
            prio = calcPrio(changedIndex,False,0)
            if reduceIfCenter[ATTRIBUTE_TYPES[prio[1]]]:
                centerReq[ATTRIBUTE_TYPES[prio[1]]] = True
            else:
                centerReq[ATTRIBUTE_TYPES[prio[2]]] = True
                
        valList = list(map(lambda t: targetVals[t],ATTRIBUTE_TYPES))
        minList = list(map(lambda t: minIfCentre[t][centerReq[t]],ATTRIBUTE_TYPES))
        
        reduceToConstrain(valList,minList,MAX_RUNES,prio)
        
        for i in range(3):
            targetList[i].value = valList[i]
        
        
        
    def requiredChanged(changedType):
        for t in ATTRIBUTE_TYPES:
            minIfCentre[t] = checkMin(required[t])
        minComment.layout = MC_HIDE
        
        classifyOptions()
        nCenterCandidates = list(prioTypes.values()).count(True)
        if nCenterCandidates == 1:
            for t in ATTRIBUTE_TYPES:
                minList[t].value = str(minIfCentre[t][prioTypes[t]])
        else:
            if list(reduceIfCenter.values()).count(True) > 1:
                for t in ATTRIBUTE_TYPES:
                    if reduceIfCenter[t]:
                        minList[t].value = str(minIfCentre[t][True])+'* ('+str(minIfCentre[t][False])+')'
                        minComment.value = MC_WRN_CENTER
                        minComment.layout = MC_SHOW
                    else:
                        minList[t].value = str(minIfCentre[t][False])
            else:
                for t in ATTRIBUTE_TYPES:
                    minList[t].value = str(minIfCentre[t][reduceIfCenter[t]])
                    
        if attributeError != ERROR['ERR_OK']:
            minComment.layout = MC_SHOW
            targetBox.layout = MC_HIDE
            if attributeError == ERROR['ERR_SING_UN_ATT_EXC']:
                t = reverseLookup(prioTypes,True)
                minComment.value = MC_ERR_ATT.format(t,MAX_PER_ATT[t])
            elif attributeError == ERROR['ERR_SUM_EXC']:
                minComment.value = MC_ERR_SUM.format(MAX_RUNES)
            else:
                err_code = reverseLoookup(ERR,attributeError)
                minComment.value = ERR_UNEXPECTED.format(err_code)
        else:
            constrainTargets(changedType)
            targetBox.layout = MC_SHOW
        
    def requirementLine(y):
        def attributeChanged(change):
            tar = change.owner
            if tar.value < 0:
                tar.value = 0
            elif tar.value > 3:
                tar.value = 3
            
            vals = list(map(lambda x: x.value,attList))
            tIndex = ATTRIBUTE_TYPES.index(tar.description) 
            prio = calcPrio(tIndex,False,0)
            reduceToConstrain(vals,[0,0,0],3,prio)
            for i in range(3):
                attList[i].value = vals[i]
            for x in range(3):
                required[ATTRIBUTE_TYPES[x]][y] = attList[x].value
            requiredChanged(tar.description)
        def skillAttribute(desc):
            sel = ipw.IntText(
                value=0,
                description=desc,
                layout=ipw.Layout(width='150px')
            )
            sel.observe(attributeChanged,names='value')
            return sel
        attList = [skillAttribute(d) for d in ATTRIBUTE_TYPES]
        sel = ipw.HBox([ipw.Label('Nº '+str(y+1))]+attList)
        return sel
    def targetLine():
        def targetChanged(change):
            tar = change.owner
            t = tar.description
            if attributeError == ERROR['ERR_OK'] and tar.value != targetVals[t]:
                targetVals[t] = tar.value
                constrainTargets(t)
        def targetAttribute(desc):
            sel = ipw.IntText(
                value=0,
                description=desc,
                layout=ipw.Layout(width='150px')
            )
            sel.observe(targetChanged,names='value')
            return sel
        return [targetAttribute(d) for d in ATTRIBUTE_TYPES]
    def minReq(desc):
        return ipw.Text(
            value='0',
            description=desc,
            layout=ipw.Layout(width='150px'),
            disabled=True
        )
    def att2text(t):
        if t == 'Atk':
            s = 'A'
        elif t == 'Def':
            s = 'D'
        else:
            s = 'B'
        return s
    
    def drawAtt(t,x,y):
        runeCanvas.stroke_text(att2text(t),x-3,y+3)
        
    def drawSkill(i,x,y):
        l = req2List(i)
        x -= 3+2*sum(l)
        s = ''
        ti = 0
        for i in range (3):
            while ti < 3:
                if l[ti] > 0:
                    l[ti] -= 1
                    s += att2text(ATTRIBUTE_TYPES[ti])
                    break
                else:
                    ti += 1
            if ti == 3:
                s += '*'*(3-i) 
                break
        
        d = 10
        runeCanvas.stroke_text(s,x,y+3)
        
    def drawSingle(l,x,y):
        if sum(l) == 0:
            s = '*'
        else:
            s = att2text(ATTRIBUTE_TYPES[l.index(1)])
        runeCanvas.stroke_text(s,x-3,y+3)
    
        
    def drawBase():
        def hexa(canvas,x,y,l,paths):
            l2 = l/2
            l3 = l2 * math.sqrt(3)
            canvas.begin_path()
            canvas.move_to(x,y-l)
            canvas.line_to(x+l3,y-l2)
            canvas.line_to(x+l3,y+l2)
            canvas.line_to(x,y+l)
            canvas.line_to(x-l3,y+l2)
            canvas.line_to(x-l3,y-l2)
            canvas.line_to(x,y-l)
            if paths:
                canvas.line_to(x,y-l*1.5)
                canvas.move_to(x+l3,y+l2)
                canvas.line_to(x+l3*1.5,y+l2*1.5)
                canvas.move_to(x-l3,y+l2)
                canvas.line_to(x-l3*1.5,y+l2*1.5)
            canvas.stroke()
        d = 10
        d3 = math.sqrt(3) * d
        p = 2*d+50
        for row in range(4):
            y = p + row * 12 * d
            x_off = p + (3-row) * 4 * d3
            for col in range(1+row):
                x = x_off + col * 8 * d3
                hexa(runeCanvas,x,y,d*2,False)
                if row < 3:
                    hexa(runeCanvas,x,y+8*d,d*4,True)
        
    
    def calc(ev):
        targetComment.value = TC_CALC
        ok, center,inner, double, outer, single = calc_main()
        if ok:
            with hold_canvas(runeCanvas):
                runeCanvas.clear()
                drawBase()
                d = 10
                d3 = math.sqrt(3) * d
                p = 2*d+50
                drawAtt(center,p+12*d3,p+24*d)
                drawSkill(inner[0],p+16*d3,p+20*d)
                drawSkill(inner[1],p+12*d3,p+32*d)
                drawSkill(inner[2],p+8*d3,p+20*d)
                drawAtt(double[0],p+16*d3,p+12*d)
                drawAtt(double[1],p+20*d3,p+24*d)
                drawAtt(double[2],p+16*d3,p+36*d)
                drawAtt(double[3],p+8*d3,p+36*d)
                drawAtt(double[4],p+4*d3,p+24*d)
                drawAtt(double[5],p+8*d3,p+12*d)
                drawSkill(outer[0],p+12*d3,p+8*d)
                drawSkill(outer[1],p+20*d3,p+32*d)
                drawSkill(outer[2],p+4*d3,p+32*d)
                drawSingle(single[0],p+12*d3,p)
                drawSingle(single[1],p+24*d3,p+36*d)
                drawSingle(single[2],p,p+36*d)
                        
                        
            targetComment.value = TC_READY
        else:
            targetComment.value = TC_FAIL
            
    
    
    def calc_main():
        ok = False
        center_candidates = calc_center()
        for center in center_candidates:
            inner_candidates = calc_inner(center)
            for inner in inner_candidates:
                double_candidates = calc_double(center,inner)
                for double in double_candidates:
                    ok, outer, single  = calc_outer(center,inner,double)
                    if ok:
                        break
                if ok:
                    break
            if ok:
                break
        return ok, center,inner, double, outer, single
                    
            
    def calc_center():
        for t in ATTRIBUTE_TYPES:
            if targetVals[t] < minIfCentre[t][False]:
                return [t]
        return list(filter(lambda t: prioTypes[t],ATTRIBUTE_TYPES))
    
    def calc_inner(center):
        wildcards = MAX_RUNES - sum(list(targetVals.values()))
        if targetVals[center] == 0 and wildcards == 0:
            return []
        
        reqAsList = [req2List(x) for x in range(6)]
        candidates = list(filter(lambda x: tryTake(reqAsList[x],center,3)[0], range(6)))
        nCand = len(candidates)
        if nCand < 3:
            return []
        inner_candidates = []
        for a in range(nCand-2):
            adj_a = wildcards + 3 - sum(reqAsList[a])
            for b in range(a+1,nCand-1):
                adj_b = adj_a + 3 - sum(reqAsList[b])
                for c in range(b+1,nCand):
                    adj_c = adj_b + 3 - sum(reqAsList[c])
                    ok = True
                    for t in ATTRIBUTE_TYPES:
                        adj = adj_c - (1 if t == center else 0)
                        candidate_a = candidates[a]
                        candidate_b = candidates[b]
                        candidate_c = candidates[c]
                        if sum(required[t]) > targetVals[t] + required[t][candidate_a] + required[t][candidate_b] + required[t][candidate_c] + adj:
                            ok = False
                    if ok:
                        inner_candidates += [[candidate_a,candidate_b,candidate_c]]
        return inner_candidates
    
    def calc_double(center,inner):
        def get_double_candidates(i):
            d_candidates = []
            def add_candidate(v):
                nonlocal d_candidates
                d_candidates += [v]
                if v[0] != v[1]:
                    v2 = v.copy()
                    v2.reverse()
                    d_candidates += [v2]
            x = inner[i]
            d = []
            for t in ATTRIBUTE_TYPES:
                d += [t]*required[t][x]
            if required[center][x] > 0:
                d.remove(center)
            elif len(d) == 3:
                return []
            if len(d) == 2:
                add_candidate(d)
            elif len(d) == 1:
                for t in ATTRIBUTE_TYPES:
                    add_candidate([t,d[0]])
            else:
                for t0 in ATTRIBUTE_TYPES:
                    for t1 in ATTRIBUTE_TYPES:
                        d_candidates += [[t0,t1]]
            return d_candidates
                
                
        candidate_slices = list(map(get_double_candidates,range(3)))
        double_candidates = []
        if not any(map(lambda x: len(candidate_slices[x]) == 0,range(2))):
            for a in candidate_slices[0]:
                for b in candidate_slices[1]:
                    for c in candidate_slices[2]:
                        double_candidates += [a+b+c]
        return double_candidates
    
    def calc_outer(center,inner,double):
        wildcards = MAX_RUNES - sum(list(targetVals.values()))
        o_candidate = []
        i = 0
        for x in range(6):
            if i < 3 and inner[i] == x:
                i += 1
            else:
                o_candidate += [x]
        l_candidate = [req2List(x) for x in o_candidate]
        for a in range(3):
            ok, a_s = tryTake(l_candidate[a],double[0],3)
            if not ok:
                continue
            ok, a_s = tryTake(a_s,double[5],2)
            if not ok:
                continue
            for b in range(3):
                if a == b:
                    continue
                ok, b_s = tryTake(l_candidate[b],double[1],3)
                if not ok:
                    continue
                ok, b_s = tryTake(b_s,double[2],2)
                if not ok:
                    continue
                c = 3 - a - b
                ok, c_s = tryTake(l_candidate[c],double[3],3)
                if not ok:
                    continue
                ok, c_s = tryTake(c_s,double[4],2)
                if not ok:
                    continue
                excess = 0
                for i in range(3):
                    t = ATTRIBUTE_TYPES[i]
                    x = 1 if t==center else 0
                    x += double.count(t)
                    x += a_s[i] + b_s[i] + c_s[i]
                    if x > MAX_PER_ATT[t]:
                        ok = False
                        break
                    excess += x - targetVals[t] if x > targetVals[t] else 0
                if ok and excess <= wildcards:
                    break
                else:
                    ok = False
            if ok:
                break
        if ok:
            return True, [o_candidate[a],o_candidate[b],o_candidate[c]], [a_s,b_s,c_s]        
        else:
            return False,[],[]
        
    skillBox = ipw.VBox([requirementLine(x) for x in range(6)])
    minList = dict((d,minReq(d)) for d in ATTRIBUTE_TYPES)
    minComment = ipw.Text(
            layout=MC_HIDE,
            disabled=True
        )
    minBox = ipw.VBox([ipw.HBox(list(minList.values())),minComment])
    targetList = targetLine()
    targetComment = ipw.Text(
            disabled=True,
            layout = ipw.Layout(width='650px'),
            value = TC_READY
        )
    calcButton = ipw.Button(description = 'Calcular')
    calcButton.on_click(calc)
    targetBox = ipw.VBox([
            ipw.HBox(targetList+[calcButton]),
            targetComment],
            layout = MC_SHOW
        )
    runeCanvas = Canvas(width=520, height=460)
    with hold_canvas(runeCanvas):
        drawBase()
                
    return skillBox,minBox,targetBox,runeCanvas
sb,mb,tb,rc = main()

## Requisitos de activación de las runas de skill

En cada fila indicar el número de runas de atributo de cada tipo requeridas para la activación (no indicar los requistos any).

In [12]:
sb

## Runas de atributo mínimas

Se deben colocar al menos este número de runas de atributo de este tipo para poder activar todas las runas de skill.

In [13]:
mb

Advertencia: Estos minimos son una condición necesaria, pero no suficiente. No son frecuentes, pero hay combinaciones que cumplen con estos mínimos que no son resolubles.

## Resolución del puzle

Indicar el numero de runas de atributo de cada tipo que se quieren colocar.

In [14]:
tb

UnboundLocalError: local variable 'inner' referenced before assignment

In [6]:
rc

Canvas(height=460, width=520)